# Scraping Ecotur

In [1]:
# Importamos librerías
from bs4 import BeautifulSoup
import requests
import pandas as pd
import html
import numpy as np
import pandas as pd
from functools import reduce


In [4]:
#pages = np.arange(1, 7) - ya no tienen distintas paginas

pages = np.arange(1, 4)
count = 1
lista_casas = []

for page in pages:
    
    count_casa = 1 #Para el print
    URL = 'https://www.ecotur.es/v_portal/busqueda/index.asp?provincia=&entornobs=&serviciosbs=&npag=' + str(page) + '&sellosbs=287'
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, 'lxml')
    
    # Guardamos lista de casas y recorremos
    casas_grid = soup.find_all(class_='fot_min')
        
    for casa in casas_grid:

        print('Casa {} de {}, pag {}/{}'.format(
            count_casa, len(casas_grid), page, len(pages)
        ))

        URL_casas = 'https://www.ecotur.es' + casa.find('a')['href']
        r = requests.get(URL_casas)
        soup_casa = BeautifulSoup(r.text, 'lxml')

        # Definimos id para cada cerveza
        id_casa = 'casa_' + str(count)

        # Nombre
        name = soup_casa.find(class_='titulo blq fs_21_32 mar0').text
        #print(name)

        # Direccion 
        direccion_total = soup_casa.find(class_='blq mart16 fs_14_16 lh_140').text.rsplit(' ',1)
        direccion = direccion_total[0][:-1]
        provincia = direccion_total[1]
        #print(price)

        # Categoría

        try:
            categoria = soup_casa.find(class_='bl_izq mart2 fs_12_14').text.replace('\xa0\r\n                ', '')
            categoria = categoria.replace('\r\n              ', '')
            categoria = categoria.replace('Categoría:', '')

        except:
            categoria = None

        #print(price)

        #descripcion 
        descripcion = soup_casa.find(class_='blq fs_14_18 colgen lh_140').text

        # Imagen
        div_image = soup_casa.find(class_='fot_min fot_centro marb10')

        image = 'https://www.ecotur.es' + div_image.find('img').attrs['src']
        #print(image)

        #print(ean)

       # Agregamos a la lista
        lista_casas.append([
            ('id', id_casa),
            ('name', name), 
            ('direccion', direccion),
            ('provincia', provincia),
            ('descripcion', descripcion),
            ('categoria', categoria),
            ('image', image),
            ])        


        # Pasamos al siguiente id
        count += 1
        count_casa += 1

Casa 1 de 8
Casa 2 de 8
Casa 3 de 8
Casa 4 de 8
Casa 5 de 8
Casa 6 de 8
Casa 7 de 8
Casa 8 de 8
Casa 1 de 7
Casa 2 de 7
Casa 3 de 7
Casa 4 de 7
Casa 5 de 7
Casa 6 de 7
Casa 7 de 7
Casa 1 de 6
Casa 2 de 6
Casa 3 de 6
Casa 4 de 6
Casa 5 de 6
Casa 6 de 6


In [6]:
df = pd.DataFrame([[x[0][1],
                    x[1][1],
                    x[2][1],
                    x[3][1],
                    x[4][1],
                    x[5][1],
                    x[6][1]
                   ] for x in lista_casas],
                  columns = ['id',
                             'name',
                             'direccion',
                             'provincia',
                             'descripcion',
                             'categoria',
                             'imagen'])

In [7]:
df

,id,name,direccion,provincia,descripcion,categoria,imagen
0,casa_1,Casas Rurales de Caín,"C/ Gregorio Pérez de María, 27B 24915 Caín de...",León,LEÓN. Las casas rurales ecológicas están situa...,Casa Rural de alquiler integro | Actividad en...,https://www.ecotur.es/v_portal/inc/imagen.asp?...
1,casa_2,Ecomuséu Ca l'Asturcón,"Foncalada, 26 33314 Argüero (Villaviciosa)",Asturias,Ecomuseo del Asturcón con exposiciones y activ...,"Finca familiar, bodega, almazara | Actividad ...",https://www.ecotur.es/v_portal/inc/imagen.asp?...
2,casa_3,Tenada del Monte,"C/ Cigüeña, 6 40160 Cabanillas del Monte (Tor...",Segovia,CONOCER PARA CONSERVAR,Actividad en la Naturaleza,https://www.ecotur.es/v_portal/inc/imagen.asp?...
3,casa_4,La Casa del Naturalista,Argüerín 24 33314 Argüeru,Asturias,\n\n\n,Apartamento | Casa Rural por habitaciones | ...,https://www.ecotur.es/v_portal/inc/imagen.asp?...
4,casa_5,Hotel Enoturismo Mainetes,"Paraje Mainetes, s/n 02651 Fuenteálamo",Albacete,ALBACETE. Hotel Rural ENOTURISMO MAINETES cuen...,Restaurante | Casa Rural de alquiler integro ...,https://www.ecotur.es/v_portal/inc/imagen.asp?...
5,casa_6,Finca Buenos Aires,Vereda La Julia Filandia,Quindío,"El proyecto de ""Finca Buenos Aires"" busca gene...",Actividad en la Naturaleza,https://www.ecotur.es/v_portal/inc/imagen.asp?...
6,casa_7,Vereda la Mariela,Vereda La Mariela Pijao,Quindío,DE CAMINO POR LA VEREDA LA MARIELA. Una propue...,Actividad en la Naturaleza,https://www.ecotur.es/v_portal/inc/imagen.asp?...
7,casa_8,Ecoaldea Illari,"Finca La Estrella, Vereda Rio Azul Pijao",Quindío,La Ecoaldea Illari ofrece una combinación únic...,Actividad en la Naturaleza,https://www.ecotur.es/v_portal/inc/imagen.asp?...
8,casa_9,Zona Bici,Carrera 22 No. 1817 La Ceja del Tambo,Antioquia,Zna Bici ha construido de manera colectiva con...,Actividad en la Naturaleza,https://www.ecotur.es/v_portal/inc/imagen.asp?...
9,casa_10,Rincón La Huella Verde - Habitaciones,"Calle Guatemala, 6 02434 Letur",Albacete,El Rincón La Huella Verde está ubicado en las ...,Casa Rural por habitaciones | Actividad en la...,https://www.ecotur.es/v_portal/inc/imagen.asp?...


In [8]:
direccion = list(df.direccion)

In [9]:
lat = []
lon = []
for i in direccion:
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent="Kerstin")
    location = geolocator.geocode(i)
    try:
        lat.append(location.latitude)
        lon.append(location.longitude)

    except:
        lat.append(None)
        lon.append(None)

    #time.sleep(1)

In [10]:
df = df.assign(latitude = lat)
df = df.assign(longitude = lon)

In [11]:
df.to_csv('ecotur_act.csv')

In [12]:
df

,id,name,direccion,provincia,descripcion,categoria,imagen,latitude,longitude
0,casa_1,Casas Rurales de Caín,"C/ Gregorio Pérez de María, 27B 24915 Caín de...",León,LEÓN. Las casas rurales ecológicas están situa...,Casa Rural de alquiler integro | Actividad en...,https://www.ecotur.es/v_portal/inc/imagen.asp?...,NaN,NaN
1,casa_2,Ecomuséu Ca l'Asturcón,"Foncalada, 26 33314 Argüero (Villaviciosa)",Asturias,Ecomuseo del Asturcón con exposiciones y activ...,"Finca familiar, bodega, almazara | Actividad ...",https://www.ecotur.es/v_portal/inc/imagen.asp?...,NaN,NaN
2,casa_3,Tenada del Monte,"C/ Cigüeña, 6 40160 Cabanillas del Monte (Tor...",Segovia,CONOCER PARA CONSERVAR,Actividad en la Naturaleza,https://www.ecotur.es/v_portal/inc/imagen.asp?...,40.976219,-4.032479
3,casa_4,La Casa del Naturalista,Argüerín 24 33314 Argüeru,Asturias,\n\n\n,Apartamento | Casa Rural por habitaciones | ...,https://www.ecotur.es/v_portal/inc/imagen.asp?...,43.536201,-5.469304
4,casa_5,Hotel Enoturismo Mainetes,"Paraje Mainetes, s/n 02651 Fuenteálamo",Albacete,ALBACETE. Hotel Rural ENOTURISMO MAINETES cuen...,Restaurante | Casa Rural de alquiler integro ...,https://www.ecotur.es/v_portal/inc/imagen.asp?...,NaN,NaN
5,casa_6,Finca Buenos Aires,Vereda La Julia Filandia,Quindío,"El proyecto de ""Finca Buenos Aires"" busca gene...",Actividad en la Naturaleza,https://www.ecotur.es/v_portal/inc/imagen.asp?...,4.676490,-75.695359
6,casa_7,Vereda la Mariela,Vereda La Mariela Pijao,Quindío,DE CAMINO POR LA VEREDA LA MARIELA. Una propue...,Actividad en la Naturaleza,https://www.ecotur.es/v_portal/inc/imagen.asp?...,4.294887,-75.757384
7,casa_8,Ecoaldea Illari,"Finca La Estrella, Vereda Rio Azul Pijao",Quindío,La Ecoaldea Illari ofrece una combinación únic...,Actividad en la Naturaleza,https://www.ecotur.es/v_portal/inc/imagen.asp?...,NaN,NaN
8,casa_9,Zona Bici,Carrera 22 No. 1817 La Ceja del Tambo,Antioquia,Zna Bici ha construido de manera colectiva con...,Actividad en la Naturaleza,https://www.ecotur.es/v_portal/inc/imagen.asp?...,NaN,NaN
9,casa_10,Rincón La Huella Verde - Habitaciones,"Calle Guatemala, 6 02434 Letur",Albacete,El Rincón La Huella Verde está ubicado en las ...,Casa Rural por habitaciones | Actividad en la...,https://www.ecotur.es/v_portal/inc/imagen.asp?...,NaN,NaN


[[('id', 'casa_1'),
  ('name',
   '\n\r\n                                            \tEcolabel\r\n                                            \t \r\n                                            \tUlle Gorri \r\n                                            '),
  ('direccion', '\nES-EU/051/00031\n'),
  ('telefono', '\nES-EU/051/00031\n'),
  ('web', None),
  ('imagen', 'None')],
 [('id', 'casa_2'),
  ('name',
   '\n\r\n                                            \tEcolabel\r\n                                            \t \r\n                                            \tLa Posada de San Marcos, Millán Arkwright SL\r\n                                            '),
  ('direccion', '\nES-AN/051/00001\n'),
  ('telefono', '\nES-AN/051/00001\n'),
  ('web', None),
  ('imagen', 'None')],
 [('id', 'casa_3'),
  ('name',
   '\n\r\n                                            \tEcolabel\r\n                                            \t \r\n                                            \tTAKE HOSTEL CON

In [ ]:
# La web tiene 6 páginas con casas rurales listadas
pages = np.arange(1, 14)
count = 1
lista_casas = []

for page in pages:
    
    count_casa = 1 #Para el print
    URL = 'http://www.redeuroparc.org/actividades/carta-europea-turismo-sostenible/empresas-adheridas-a-la-cets/consulta?field_cets_enp_tid=All&field_comunidades_autonomas_tid=All&title=&field_cets_actividad_tid=All&status=1&page='+ str(page)
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, 'lxml')
    
    # Guardamos lista de cervezas y recorremos
    casas_grid = soup.find_all(class_='views-field views-field-title')
    #print(len(cervezas_grid))

    for casa in casas_grid:
        
        print('Casa {} de {}, pag {}/{}'.format(
            count_casa, len(casas_grid), page, len(pages)
        ))

        URL_casas = 'http://www.redeuroparc.org' + casa.find('a')['href']
        r = requests.get(URL_casas)
        soup_casa = BeautifulSoup(r.text, 'lxml')
        
        # Definimos id para cada cerveza
        id_casa = 'casa_' + str(count)
        
        # Nombre
        name = soup_casa.find(class_='col-md-12')['H1']
        #print(name)
        
        # Direccion 
        direccion = soup_casa.find(class_='field-item even').text
        #print(price)
        
        # Categoría
        
        try:
            categoria = soup_casa.find(class_='field-label').text
        
        except:
            categoria = None

        #print(price)
        
        #Telefono 
        telefono = soup_casa.find(class_='field-item even').text
                
        # Imagen
        div_image = soup_casa.find(class_='field-item even')
        
        image = div_image.find('img').attrs['src']
        #print(image)
        
        #print(ean)
        
       # Agregamos a la lista
        lista_casas.append([
            ('id', id_casa),
            ('name', name), 
            ('direccion', direccion),
            ('telefono', telefono),
            ('categoria', categoria),
            ('image', image),
            ])        
        
        
        # Pasamos al siguiente id
        count += 1
        
        count_casa += 1
        
